IMPORT ALL THE LIBRARIES

In [15]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd
import requests
from io import StringIO
from io import BytesIO  # For handling Excel files

SET THE PATH FOR  YOUR DOOR AND SCOPE

In [3]:
# Replace with the path to your service account credentials
SERVICE_ACCOUNT_FILE = "/content/drive-bridge-financial-data-e21f827ecd9b.json"
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']


#service account file , tell the code where is the door address and what are the permission to open that door
#scope script defines the level of access (permissions) that your script is requesting from the Google API.  (Here the scope is read only.)


AUTHETICATE AND CREATE THE SERVICES

In [4]:
# Authenticate and create the service

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)  #from the service account file we are using a function credentials.from_service_account_file
#It reads the service account file and authenticates our script using the permissions (scopes) defined.


service = build('drive', 'v3', credentials=credentials)
#This line builds the Google Drive API service, allowing your script to interact with Google Drive. Here's what it does
#build() is a function provided by the googleapiclient.discovery module that initializes the API service
#The first argument, 'drive', specifies the Google Drive API you want to use.

FOLDER ID

In [5]:
# Replace with your Google Drive folder ID
FOLDER_ID = '15UYA2eHZpuVPtGsmmJ0b5g4UGEXjRZaG'

FUNCTION TO LIST ALL THE FILES IN GOOGLE DRIVE

In [9]:
def list_files(service, folder_id):
    results = service.files().list(
        q=f"'{folder_id}' in parents",
        fields="files(id, name, mimeType)"
    ).execute()
    return results.get('files', [])


In [10]:
# Fetch the files
files = list_files(service, FOLDER_ID)
files

[{'mimeType': 'application/vnd.google-apps.spreadsheet',
  'id': '1U-X2ss_mhQMQZBTiEynYwckujN-JKA7VBVxqGoTV2mI',
  'name': 'combined_part_25'},
 {'mimeType': 'application/vnd.google-apps.spreadsheet',
  'id': '1TT2UOPw92KfNj-JRBzR4JGmoyXCaEo36L_-A6kcePPA',
  'name': 'combined_part_24'},
 {'mimeType': 'application/vnd.google-apps.spreadsheet',
  'id': '1BkCewWqrJF5qJPE1WoyWpwHbr25KnGm_p5CGzAl8k1Y',
  'name': 'combined_part_22'},
 {'mimeType': 'application/vnd.google-apps.spreadsheet',
  'id': '1NN-2B3ZxanZuYf-U3DfRSO3b9MMH3pbE3G3GNR92zKg',
  'name': 'combined_part_23'},
 {'mimeType': 'application/vnd.google-apps.spreadsheet',
  'id': '1Qh9SrYHRzYVW1n6mlFlOo-S3FoB3WBCYQPLeWpMAOlY',
  'name': 'combined_part_21'},
 {'mimeType': 'application/vnd.google-apps.spreadsheet',
  'id': '1dFcuDgduCTDXtWPkM6k0MOzSPMQlWmwwWPuxz2EpOWY',
  'name': 'combined_part_20'},
 {'mimeType': 'application/vnd.google-apps.spreadsheet',
  'id': '1TXu88S7c9-0C-pZmXkUEFFTldssX8RgpmE3Yvar1WqQ',
  'name': 'combined_par

In [11]:
# List to store dataframes for file content
file_dataframes = []

for file in files:
    file_id = file['id']
    file_name = file['name']
    mime_type = file['mimeType']

    # Check if the file is Google Sheets, CSV, or Excel
    if mime_type == 'application/vnd.google-apps.spreadsheet':
        # Export Google Sheets as CSV
        download_url = f"https://docs.google.com/spreadsheets/d/{file_id}/export?format=csv"
    elif mime_type == 'text/csv':
        # If it's already a CSV file
        download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
    elif mime_type == 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet':
        # If it's an Excel file
        download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
    else:
        continue  # Skip files that are not CSV, Google Sheets, or Excel


    response = requests.get(download_url)# Download and read the CSV or Excel file into a pandas dataframe
    if response.status_code == 200:  #status code 200 means files was successfully downonloaded
        if mime_type == 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet':
            # For Excel, use BytesIO to read the binary content
            df = pd.read_excel(BytesIO(response.content))
        else:
            # For CSV and Google Sheets (converted to CSV)
            df = pd.read_csv(StringIO(response.content.decode('utf-8')))
        file_dataframes.append(df)
    else:
        print(f"Error downloading file: {file_name}")



In [12]:
file_dataframes

[           ID Customer_ID     Month            Name   Age          SSN  \
 0     0x23cca  CUS_0x8b6d   January  Scott Haggettl    23  796-69-6764   
 1     0x23ccb  CUS_0x8b6d  February  Scott Haggettl    23    #F%$D@*&8   
 2     0x23ccc  CUS_0x8b6d     March  Scott Haggettl    23  796-69-6764   
 3     0x23ccd  CUS_0x8b6d     April  Scott Haggettl  5615  796-69-6764   
 4     0x23cce  CUS_0x8b6d       May  Scott Haggettl    23  796-69-6764   
 ...       ...         ...       ...             ...   ...          ...   
 5995  0x25fe9  CUS_0x942c     April           Nicks    25  078-73-5990   
 5996  0x25fea  CUS_0x942c       May           Nicks    25  078-73-5990   
 5997  0x25feb  CUS_0x942c      June           Nicks    25  078-73-5990   
 5998  0x25fec  CUS_0x942c      July           Nicks    25  078-73-5990   
 5999  0x25fed  CUS_0x942c    August           Nicks    25  078-73-5990   
 
      Occupation Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  ...  \
 0      Musician 

In [13]:
# Combine all dataframes into a single one
if file_dataframes:
    combined_df = pd.concat(file_dataframes, ignore_index=True)

In [14]:
combined_df

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x23cca,CUS_0x8b6d,January,Scott Haggettl,23,796-69-6764,Musician,102594.9,8391.575000,3,...,Standard,416.11,30.601881,NaN,No,0.000000,653.9822978912564,Low_spent_Large_value_payments,455.175202,Standard
1,0x23ccb,CUS_0x8b6d,February,Scott Haggettl,23,#F%$D@*&8,Musician,102594.9_,8391.575000,3,...,_,416.11,43.528168,18 Years and 9 Months,No,0.000000,186.52023659780826,High_spent_Large_value_payments,892.637263,Standard
2,0x23ccc,CUS_0x8b6d,March,Scott Haggettl,23,796-69-6764,Musician,102594.9,8391.575000,3,...,Standard,416.11,37.487060,18 Years and 10 Months,NM,0.000000,250.4766071514879,High_spent_Medium_value_payments,838.680893,Standard
3,0x23ccd,CUS_0x8b6d,April,Scott Haggettl,5615,796-69-6764,_______,102594.9,8391.575000,3,...,Standard,416.11,35.125945,18 Years and 11 Months,No,0.000000,143.98347272538896,High_spent_Large_value_payments,935.174027,Standard
4,0x23cce,CUS_0x8b6d,May,Scott Haggettl,23,796-69-6764,Musician,102594.9,NaN,3,...,Standard,416.11,38.284492,19 Years and 0 Months,No,0.000000,651.6394489063042,Low_spent_Medium_value_payments,467.518051,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,0x5c45,CUS_0x8a8c,December,Karen Jacobsu,32,520-12-7956,Doctor,14219.02,NaN,10,...,Bad,3628.5,27.975839,3 Years and 8 Months,NM,76.486008,79.3613953137231,Low_spent_Medium_value_payments,236.84443,NaN
149996,0x5c4e,CUS_0x122f,September,Donovang,24,179-85-5358,Media_Manager,18231.52,NaN,6,...,_,1358.96,26.281263,15 Years and 6 Months,Yes,86.608647,154.47017169345767,Low_spent_Small_value_payments,193.750515,NaN
149997,0x5c4f,CUS_0x122f,October,Donovang,24,179-85-5358,Media_Manager,18231.52,1448.293333,6,...,Bad,1358.96,33.889744,15 Years and 7 Months,Yes,86.608647,42.358067683971626,High_spent_Medium_value_payments,265.862619,NaN
149998,0x5c50,CUS_0x122f,November,NaN,24,179-85-5358,Media_Manager,18231.52,1448.293333,6,...,_,1358.96,34.718260,15 Years and 8 Months,Yes,86.608647,172.55816847802882,Low_spent_Small_value_payments,175.662518,NaN
